# OCI Vision API – Image Analysis (Beginner Notebook)

### What this file does:
Analyze images using OCI Vision for object detection, text recognition, classification, and face detection. This walkthrough notebook is adapted from the logic in `vision/oci_vision.py`.

**Documentation to reference:**
- OCI Vision: https://docs.oracle.com/en-us/iaas/Content/vision/using/home.htm
- OCI Python SDK: https://github.com/oracle/oci-python-sdk/tree/master/src/oci/ai_vision

**Relevant slack channels:**
- #oci_ai_vision_support: *for OCI Vision API questions*
- #igiu-innovation-lab: *general discussions on your project*
- #igiu-ai-learning: *help with sandbox environment or help with running this code*

**Env setup:**
- sandbox.yaml: Contains OCI config, compartment, and bucket details.
- .env: Load environment variables if needed.
- configure cwd for jupyter match your workspace python code: 
    -  vscode menu -> Settings > Extensions > Jupyter > Notebook File Root
    -  change from `${fileDirname}` to `${workspaceFolder}`


**How to run in notebook:**
- Make sure your runtime environment has all dependencies and access to required config files.
- Run the notebook cells in order.

---

## Step 1: Setup and Requirements

**Key Concepts:**
- **Environment Setup:** Before interacting with OCI services, you need to configure your environment with credentials, compartment IDs, and bucket details. This is typically done via a YAML config file (sandbox.yaml) and environment variables.
- **Dependencies:** Install necessary libraries like the OCI SDK and configuration loaders.
- **Libraries:** Use packages for config management, environment variables, and OCI interactions.

In this step, we'll install dependencies and import required libraries.

In [ ]:
# Import necessary libraries
import os
from dotenv import load_dotenv
from envyaml import EnvYAML
from pathlib import Path
import oci
from oci.object_storage import ObjectStorageClient
from IPython.display import Image, display

# Load environment variables
load_dotenv()

print("Libraries imported and environment loaded.")

## Step 2: Load OCI/Sandbox Configuration

**Key Concepts:**
- **Configuration Loading:** Securely load settings from a config file to authenticate and specify resources without hardcoding credentials.
- **OCI Config:** Includes paths to config files, profiles, compartments, and bucket details.
- **Error Handling:** Validate that the config is loaded correctly to avoid runtime errors.

In this step, we'll load and validate the configuration.

In [ ]:
# Define paths and load configuration
# Make sure your sandbox.yaml file is set up for your environment. You might have to specify the full path depending on your `cwd`.
# You can also try making your cwd for jupyter match your workspace python code:
# vscode menu -> Settings > Extensions > Jupyter > Notebook File Root
# change from ${fileDirname} to ${workspaceFolder}

SANDBOX_CONFIG_FILE = "sandbox.yaml"
DEFAULT_FILE = Path("./vision/receipt.png")

def load_config(config_path):
    try:
        return EnvYAML(config_path)
    except FileNotFoundError:
        print(f"Error: Configuration file '{config_path}' not found.")
        return None
    except Exception as e:
        print(f"Error loading config: {e}")
        return None

scfg = load_config(SANDBOX_CONFIG_FILE)
assert scfg is not None and 'oci' in scfg and 'bucket' in scfg, "Check your sandbox.yaml config!"
oci_cfg = oci.config.from_file(os.path.expanduser(scfg["oci"]["configFile"]), scfg["oci"]["profile"])
bucket_cfg = scfg["bucket"]
compartment_id = scfg["oci"]["compartment"]

print("Configuration loaded successfully.")

## Step 3: Select and Display an Image

**Key Concepts:**
- **Image Selection:** Choose the image file to analyze. Ensure it's in a supported format (e.g., JPEG, PNG).
- **Display:** Visualize the image to confirm it's the correct one before processing.
- **Path Handling:** Use Path objects for robust file path management.

In this step, we'll select the image and display it.

In [ ]:
# Select the image file
# Change this if you want to use another image:
FILE_PATH = Path("vision/receipt.png")
# Example: FILE_PATH = Path("vision/dussera-b.jpg")

print(f"Selected image: {FILE_PATH}")

# Display the image
display(Image(filename=str(FILE_PATH)))

## Step 4: Upload Image to OCI Object Storage

**Key Concepts:**
- **Object Storage:** OCI's scalable storage for files. Images must be uploaded here for analysis.
- **Bucket and Namespace:** Organize files in buckets within a namespace for access control.
- **Prefix:** Use prefixes to categorize objects, similar to folders.

In this step, we'll upload the selected image to Object Storage.

In [ ]:
# Function to upload file to Object Storage
def upload(oci_cfg, bucket_cfg, file_path):
    if not file_path.exists():
        print(f"Error: File '{file_path}' not found.")
        return False
    object_storage_client = ObjectStorageClient(oci_cfg)
    print(f"Uploading file {file_path} ...")
    object_storage_client.put_object(
        bucket_cfg["namespace"], 
        bucket_cfg["bucketName"], 
        f"{bucket_cfg['prefix']}/{file_path.name}", 
        open(file_path, 'rb')
    )
    print("Upload completed!")
    return True

# Perform the upload
uploaded = upload(oci_cfg, bucket_cfg, FILE_PATH)
if not uploaded:
    raise ValueError("Upload failed. Check your image and configuration.")
else:
    print("Image uploaded successfully to Object Storage.")

## Step 5: Set Up Analysis Features

**Key Concepts:**
- **Vision Service Features:** OCI Vision can perform multiple analyses: classification (e.g., image type), object detection, text detection (OCR), and face detection.
- **Feature Selection:** Choose which analyses to run based on your needs to optimize cost and performance.
- **Request Details:** Specify the image location and compartment for processing.

In this step, we'll define the features and prepare the analysis request.

In [ ]:
# Define helper functions for image location and analysis details
def get_image_location(bucket_cfg, file_name):
    image = oci.ai_vision.models.ObjectStorageImageDetails(
        source="OBJECT_STORAGE",
        namespace_name=bucket_cfg["namespace"],
        bucket_name=bucket_cfg["bucketName"],
        object_name=f"{bucket_cfg['prefix']}/{file_name}"
    )
    return image

def get_analyze_image_details(features, compartment_id, bucket_cfg, file_name):
    details = oci.ai_vision.models.AnalyzeImageDetails(
        features=features,
        image=get_image_location(bucket_cfg, file_name),
        compartment_id=compartment_id
    )
    return details

# Configure features: classification, object detection, text detection, face detection
features = [
    oci.ai_vision.models.ImageClassificationFeature(),
    oci.ai_vision.models.ImageObjectDetectionFeature(),
    oci.ai_vision.models.ImageTextDetectionFeature(),
    oci.ai_vision.models.FaceDetectionFeature()
]

print("Analysis features configured.")

## Step 6: Analyze Image

**Key Concepts:**
- **Synchronous Analysis:** OCI Vision processes images in real-time via API calls.
- **Client Initialization:** Create a client with your OCI config to interact with the service.
- **Response Handling:** Capture the analysis results, including status and data.

In this step, we'll send the image for analysis and receive the results.

In [ ]:
# Initialize the Vision client
vision_client = oci.ai_vision.AIServiceVisionClient(config=oci_cfg)

# Perform the analysis
try:
    response = vision_client.analyze_image(
        get_analyze_image_details(features, compartment_id, bucket_cfg, FILE_PATH.name)
    )
    if response:
        print("Analysis complete! (Status: ", response.status, ")")
    else:
        print("No response received.")
        response = None
except Exception as e:
    print(f"Error during analysis: {e}")
    response = None

## Step 7: Parse and Print Results

**Key Concepts:**
- **Result Parsing:** Extract and display insights from the response, such as classifications, detected objects, text, and faces.
- **Confidence Scores:** Each detection includes a confidence level to gauge reliability.
- **Structured Output:** Present data in a readable format for easy interpretation.

In this step, we'll parse the analysis results and display them.

In [ ]:
# Function to parse and display vision response
def parse_vision_response(data):
    print("\nOCI Vision Analysis Results:")
    labels = getattr(data, 'labels', None)
    if labels:
        print("\nClassifications:")
        for label in labels:
            print(f"  - {label.name}: {label.confidence:.2f}")
    image_objects = getattr(data, 'image_objects', None)
    if image_objects:
        print("\nDetected Objects:")
        for obj in image_objects:
            print(f"  - {obj.name} : {obj.confidence:.2f}")
    image_text = getattr(data, 'image_text', None)
    if image_text and hasattr(image_text, 'lines'):
        print("\nDetected Text Lines:")
        for line in image_text.lines:
            print(f"  - {line.text}: {line.confidence:.2f}")
    detected_faces = getattr(data, 'detected_faces', None)
    if detected_faces:
        print("\nDetected Faces:")
        for face in detected_faces:
            print(f"  - Face confidence: {face.confidence:.2f} : Quality {face.quality_score:.2f}")
    if not any([labels, image_objects, image_text, detected_faces]):
        print("No features detected in the image.")

# Parse the results
if response is not None and hasattr(response, 'data'):
    parse_vision_response(response.data)
else:
    print("No results to parse.")

print("\nParsing complete.")

## 🎉 Done! Next Steps

- Try different images or text files for analysis.
- Adapt feature selection for narrower use-cases (object-detection only, for instance).
- Explore downstream automation: parse response dicts for structured results!

## 🧑‍💻 Project Ideas for Practice

Below are some practical project prompts. Try one (or all) after you run a basic image through the models!

1. **OCR a Receipt**: Extract text from receipts and parse into structured data (date, amount, items).
2. **Document Type Identification**: Classify uploaded documents (e.g., receipts, invoices, IDs).
3. **ID Verification**: Confirm ID matches the user based on name and DOB from detected text.
4. **Accessibility Tool**: Describe images for visually impaired users using object detection and text.
5. **Content Moderation**: Detect inappropriate objects or text in images for automated filtering.

If you have OCI API errors, check your config, permissions, and credentials.